# wrangling motion and censor regressors for use with afni functions

In [1]:
from bids.layout import BIDSLayout
import pandas as pd
import os, re, json

First I define paramaters and collect the subject/session iterables:

In [3]:
BIDSdir = "/scratch/qbi/uqkgarn1/STRIWP1/"
TRs_single_echo = pd.Series(str(x) for x in [1510, 1920])
TRs_multi_echo = pd.Series(str(700))
echoes = pd.Series(str(x) for x in [1, 2])
layout = BIDSLayout(BIDSdir)
subs = layout.get_subjects() # BIDSLayout does not appear to detect anything below the session folders, despite them being 
# BIDS compliant. However, the BIDSDataGrabber does appear to find the files, so pursuing for now
sess = layout.get_sessions()
subs

['01', '02', '03', '04', '05']

Now, for each subject and session, I need to create two files required by the AFNI 3dtProject function. One containing the censor data, the other containing the motion regressors. 

In [3]:
# first get a list of all the confounds files for the single echo TRs to which I will apply the printCensored and printMotion functions below
fstr_sing_echo = ''.join([BIDSdir, 'derivatives/sub-{0}/ses-{1}/func/sub-{0}_ses-{1}_task-learnAtt_acq-TR{2}_desc-confounds_regressors.tsv'])
fstr_multi_echo = ''.join([BIDSdir, 'derivatives/sub-{0}/ses-{1}/func/sub-{0}_ses-{1}_task-learnAtt_acq-TR{2}_echo-{3}_desc-confounds_regressors.tsv'])
fns = [fstr_sing_echo.format(subnum, sesnum, TRnum) for subnum in subs for sesnum in sess for TRnum in TRs_single_echo] + [fstr_multi_echo.format(subnum, sesnum, TRnum, eNum) for subnum in subs for sesnum in sess for TRnum in TRs_multi_echo for eNum in echoes]
fns
# rfns is a list of non-existent files, for example the subject did not complete the session, which I then exclude from the list of filenames
rfns = ['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/ses-04/func/sub-01_ses-04_task-learnAtt_acq-TR1510_desc-confounds_regressors.tsv', # didn't complete session
        '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/ses-04/func/sub-01_ses-04_task-learnAtt_acq-TR1920_desc-confounds_regressors.tsv', # didn't complete session
        '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/ses-04/func/sub-01_ses-04_task-learnAtt_acq-TR700_echo-1_desc-confounds_regressors.tsv', # didn't complete session
        '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/ses-04/func/sub-01_ses-04_task-learnAtt_acq-TR700_echo-2_desc-confounds_regressors.tsv', # didn't complete session
        '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-03/ses-04/func/sub-03_ses-04_task-learnAtt_acq-TR1920_desc-confounds_regressors.tsv'] # error constructing scan
fns_list = [e for e in fns if e not in rfns]
fns_list

['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/ses-02/func/sub-01_ses-02_task-learnAtt_acq-TR1510_desc-confounds_regressors.tsv',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/ses-02/func/sub-01_ses-02_task-learnAtt_acq-TR1920_desc-confounds_regressors.tsv',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/ses-03/func/sub-01_ses-03_task-learnAtt_acq-TR1510_desc-confounds_regressors.tsv',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/ses-03/func/sub-01_ses-03_task-learnAtt_acq-TR1920_desc-confounds_regressors.tsv',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-02/ses-02/func/sub-02_ses-02_task-learnAtt_acq-TR1510_desc-confounds_regressors.tsv',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-02/ses-02/func/sub-02_ses-02_task-learnAtt_acq-TR1920_desc-confounds_regressors.tsv',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-02/ses-03/func/sub-02_ses-03_task-learnAtt_acq-TR1510_desc-confounds_regressors.tsv',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-02/ses-03/func

In [4]:
def printNewJson(fname, data): # function to print json file, given fname (str) and data = {}
    with open(fname, 'w') as outfile:
        json.dump(data, outfile)
        
def updateJson(json_fname, update_data):
    with open(json_fname, "r+") as file:
        data = json.load(file)
        data.update(update_data)
        file.seek(0)
        json.dump(data, file)

In [5]:
def printCensored(fname): # fname = confounds from get_confounds
    import pandas as pd
    import re
#fname = fns_list[-1] # for testing
    # set filename for output file
    censfname = fname.replace('confounds', 'censor')
    censfname = censfname.replace('tsv', '1D')

    # read in data, norm and set censored timepoints
    data = pd.read_csv(fname, sep='\t', usecols=['dvars'])
    normd = (data-data.mean())/data.std()
    normd = normd.abs() < 1.5
    normd.dvars.astype(int)
    normd = (normd*1)
    normd.to_csv(censfname, sep='\t', index=False, header=False)   
#     # now write the json file to accompany it
    censfname_json = censfname.replace('1D', 'json')
    TR = re.search('.*acq-TR([0-9]*)_.*', censfname_json)
    echo = re.search('.*echo-([0-9]*)_.*', censfname_json);
    if not re.search('.*echo-([0-9]*)_.*', censfname_json):
           cens_json_data = {"tsvType":"AFNI-censor", "TR":str(int(TR.group(1))/1000)}
    else: 
           cens_json_data = {"tsvType":"AFNI-censor", "TR":str(int(TR.group(1))/1000), 
                             "echo": echo.group(1)}
    
    printNewJson(censfname_json, cens_json_data)
    return censfname
#censfname

In [10]:
def printMotion(fname): # fname = confounds from get_confounds
    import pandas as pd
    import re 
    data = pd.read_csv(fname, sep='\t', usecols=['trans_x', 'trans_x_derivative1',
                                                 'trans_y', 'trans_y_derivative1',
                                                 'trans_z', 'trans_z_derivative1',
                                                 'rot_x', 'rot_x_derivative1',
                                                 'rot_y', 'rot_y_derivative1',
                                                 'rot_z', 'rot_z_derivative1'])
    data = data.fillna(value=0)
    
    savefname = fname.replace('confounds', 'motion')
    savefname = savefname.replace('tsv', 'txt')
    data.to_csv(savefname, sep=' ', index=False, header=False)
    # write json file to accompany
    motionfname_json = savefname.replace('txt', 'json')
    TR = re.search('.*acq-TR([0-9]*)_.*', motionfname_json)
    echo = re.search('.*echo-([0-9]*)_.*', motionfname_json)
    if not re.search('.*echo-([0-9]*)_.*', motionfname_json):
        motion_json_data = {"tsvType":"AFNI-motion", "TR":str(int(TR.group(1))/1000)}
    else:         
        motion_json_data = {"tsvType":"AFNI-motion", "TR":str(int(TR.group(1))/1000), 
                            "echo": echo.group(1)}
    printNewJson(motionfname_json, motion_json_data)
    return savefname

In [11]:
cenf = [printCensored(f) for f in fns_list]
motf = [printMotion(f) for f in fns_list]

In [ ]:
fname = '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/ses-02/func/sub-01_ses-02_task-learnAtt_acq-TR1510_desc-confounds_regressors.tsv'

In [ ]:
    data = pd.read_csv(fname, sep='\t', usecols=['trans_x', 'trans_x_derivative1',
                                                 'trans_y', 'trans_y_derivative1',
                                                 'trans_z', 'trans_z_derivative1',
                                                 'rot_x', 'rot_x_derivative1',
                                                 'rot_y', 'rot_y_derivative1',
                                                 'rot_z', 'rot_z_derivative1'])

In [ ]:
data = data.fillna(value=0)
data

## manipulating T2 json files to make queries using BIDSDataGrabber

In [ ]:
# first get a list of all the confounds files for the single echo TRs to which I will apply the printCensored and printMotion functions below
T2fstr_sing_echo = ''.join([BIDSdir, 'derivatives/sub-{0}/ses-{1}/func/sub-{0}_ses-{1}_task-learnAtt_acq-TR{2}_space-T1w_desc-preproc_bold.json'])
T2fstr_multi_echo = ''.join([BIDSdir, 'derivatives/sub-{0}/ses-{1}/func/sub-{0}_ses-{1}_task-learnAtt_acq-TR{2}_echo-{3}_space-T1w_desc-preproc_bold.json'])
#T2fns = [T2fstr_sing_echo.format(subnum, sesnum, TRnum) for subnum in subs for sesnum in sess for TRnum in TRs_single_echo] + [T2fstr_multi_echo.format(subnum, sesnum, TRnum, eNum) for subnum in subs for sesnum in sess for TRnum in TRs_multi_echo for eNum in echoes]
T2fns_singEcho = [T2fstr_sing_echo.format(subnum, sesnum, TRnum) for subnum in subs for sesnum in sess for TRnum in TRs_single_echo]
T2fns_multiEcho = [T2fstr_multi_echo.format(subnum, sesnum, TRnum, eNum) for subnum in subs for sesnum in sess for TRnum in TRs_multi_echo for eNum in echoes]
T2fns_multiEcho

T2rfns_singEcho = ['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/ses-04/func/sub-01_ses-04_task-learnAtt_acq-TR1510_space-T1w_desc-preproc_bold.json', # didn't complete session
                   '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/ses-04/func/sub-01_ses-04_task-learnAtt_acq-TR1920_space-T1w_desc-preproc_bold.json', # didn't complete session]
                   '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-03/ses-04/func/sub-03_ses-04_task-learnAtt_acq-TR1920_space-T1w_desc-preproc_bold.json']
                   # remove nonexistent files
T2rfns_multiEcho = ['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/ses-04/func/sub-01_ses-04_task-learnAtt_acq-TR700_echo-1_space-T1w_desc-preproc_bold.json', # didn't complete session
                    '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/ses-04/func/sub-01_ses-04_task-learnAtt_acq-TR700_echo-2_space-T1w_desc-preproc_bold.json']
    
# rfns is a list of non-existent files, for example the subject did not complete the session, which I then exclude from the list of filenames
# T2rfns = ['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/ses-04/func/sub-01_ses-04_task-learnAtt_acq-TR1510_space-T1w_desc-preproc_bold.json', # didn't complete session
#           '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/ses-04/func/sub-01_ses-04_task-learnAtt_acq-TR1920_space-T1w_desc-preproc_bold.json', # didn't complete session
#           '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/ses-04/func/sub-01_ses-04_task-learnAtt_acq-TR700_echo-1_space-T1w_desc-preproc_bold.json', # didn't complete session
#           '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/ses-04/func/sub-01_ses-04_task-learnAtt_acq-TR700_echo-2_space-T1w_desc-preproc_bold.json', # didn't complete session
#           '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-03/ses-04/func/sub-03_ses-04_task-learnAtt_acq-TR1920_space-T1w_desc-preproc_bold.json'] # error constructing scan

T2fns_singEcho_list = [e for e in T2fns_singEcho if e not in T2rfns_singEcho]
T2fns_multiEcho_list = [e for e in T2fns_multiEcho if e not in T2rfns_multiEcho]
# T2fns_list
T2fns_multiEcho_list


In [ ]:
singEcho_update_data = {"space":"T1w"}
for f in T2fns_singEcho_list: 
    updateJson(f, singEcho_update_data) 

In [ ]:
for f in T2fns_multiEcho_list:
    echo = re.search('.*echo-([0-9]*)_.*', f)
    echo = echo.group(1)
    update_data = {"space":"T1w", "echo": echo }
    updateJson(f, update_data)